<div style="max-width:66ch;">

# Lecture notes - Hich performance Pandas

This is a lecture note on **high performance Pandas** - but it's built upon contents from pandas and previous course:

- Python programming

<p class = "alert alert-info" role="alert"><b>Note</b> that this lecture note gives a brief introduction to high performance. I encourage you to read further about high performance.

Read more

- [Enhancing performance](https://pandas.pydata.org/docs/user_guide/enhancingperf.html)
- [pandas eval()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.eval.html?highlight=eval#pandas.DataFrame.eval)
- [pandas query](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html?highlight=query#pandas.DataFrame.query)
- [Scaling to large datasets](https://pandas.pydata.org/docs/user_guide/scale.html?highlight=efficency)


</div>


<div style="max-width:66ch;">

## Eval

We use a compound expression to motivation eval(): 

```python
mask = (x > 0.5) & (y < 0.5)
```
will create the following steps which are explicitly allocated in memory: 

```python
tmp1 = (x > 0.5)
tmp2 = (y < 0.5)
mask = tmp1 & tmp2
```

Using eval() will perform elementwise directly without intermediate steps using numexpr. 

eval can be slower than normal pandas expressions. Rule of thumb:
if df rows > 10000 can use eval() else use normal df expressions


Note that normal Python's eval can be a security risk if used together with user input. Pandas eval however can't execute arbitrary functions. 

</div>

In [1]:
import numpy as np 
import pandas as pd 

nrows, ncols = 1000000, 100

df1, df2, df3, df4 = [pd.DataFrame(np.random.randn(nrows, ncols)) for _ in range(4)]

In [2]:
# pd.eval()
%timeit sum_plain = df1+df2+df3+df4
%timeit sum_eval = pd.eval("df1 + df2 + df3 + df4")
sum_plain = df1+df2+df3+df4
sum_eval = pd.eval("df1 + df2 + df3 + df4")
sum_plain.equals(sum_eval)

1.74 s ± 47.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.79 s ± 148 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


True

In [3]:
# df.eval()
rolls = pd.DataFrame(np.random.randint(1,6,(6,3)), columns = ["Die1", "Die2", "Die3"])
rolls.eval("Sum = Die1 + Die2 + Die3", inplace = True)
rolls

,Die1,Die2,Die3,Sum
0,1,3,1,5
1,1,4,1,6
2,4,5,5,14
3,4,2,4,10
4,3,2,1,6
5,1,5,2,8


In [4]:
# use variables
high = 10 
rolls.eval("High = Sum > @high", inplace = True)
rolls

,Die1,Die2,Die3,Sum,High
0,1,3,1,5,False
1,1,4,1,6,False
2,4,5,5,14,True
3,4,2,4,10,False
4,3,2,1,6,False
5,1,5,2,8,False


<div style="max-width:66ch;">

## Query

Cleaner syntax for selection. Faster for larger datasets and compound expressions.

</div>

In [5]:
low = 10
small_plain = rolls[rolls["Sum"] < low]
small_plain

,Die1,Die2,Die3,Sum,High
0,1,3,1,5,False
1,1,4,1,6,False
4,3,2,1,6,False
5,1,5,2,8,False


In [6]:
small_query = rolls.query("Sum < @low")
small_query

,Die1,Die2,Die3,Sum,High
0,1,3,1,5,False
1,1,4,1,6,False
4,3,2,1,6,False
5,1,5,2,8,False


In [7]:
os = pd.read_csv("data/athlete_events.csv")
os.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [8]:
%timeit os[os["Season"] == "Winter"]
%timeit os.query("Season == 'Winter'")

plain = os[os["Season"] == "Winter"]
query = os.query("Season == 'Winter'")

plain.equals(query)

28.4 ms ± 4.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
12 ms ± 404 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


True

In [9]:
%timeit os[os["Height"] > 180]
%timeit os.query("Height > 180") # note that query is slower here

plain = os[os["Height"] > 180]
query = os.query("Height > 180") 

plain.equals(query)

7.4 ms ± 56.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
8.97 ms ± 236 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


True

In [10]:
# query faster on compound expressions as it doesn't have to save intermediate results into memory
%timeit os[(os["Sex"] == "F") & (os["Height"] > 180) & (os["NOC"] == "SWE")]
%timeit os.query("Sex == 'F' & Height > 180 & NOC == 'SWE'") 

plain = os[(os["Sex"] == "F") & (os["Height"] > 180) & (os["NOC"] == "SWE")]
query = os.query("Sex == 'F' & Height > 180 & NOC == 'SWE'")

plain.equals(query)

32.4 ms ± 65.9 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
12.3 ms ± 41.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


True

<div style="max-width:66ch;">

## Using pyarrow backend

As of pandas version before 2.0 we had only numpy for storing data in the backend. However since 2.0 we can change the backend to pyarrow. So the motivation behind pyarrow is that usually when moving arrays between programming languages such as Python and C which numpy is built upon usually requires some computational overhead. Pyarrow however is a way of storing arrays in memory in a programming agnostic way removing the computational overhead. 

Normal pandas operators and methods seems to beat eval and query when using pyarrow backend for those tests we've performed. 

Note that you have to install pyarrow for this to work 

</div>

In [12]:
os = os.convert_dtypes(dtype_backend="pyarrow")
os.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype          
---  ------  --------------   -----          
 0   ID      271116 non-null  int64[pyarrow] 
 1   Name    271116 non-null  string[pyarrow]
 2   Sex     271116 non-null  string[pyarrow]
 3   Age     261642 non-null  int64[pyarrow] 
 4   Height  210945 non-null  int64[pyarrow] 
 5   Weight  208241 non-null  double[pyarrow]
 6   Team    271116 non-null  string[pyarrow]
 7   NOC     271116 non-null  string[pyarrow]
 8   Games   271116 non-null  string[pyarrow]
 9   Year    271116 non-null  int64[pyarrow] 
 10  Season  271116 non-null  string[pyarrow]
 11  City    271116 non-null  string[pyarrow]
 12  Sport   271116 non-null  string[pyarrow]
 13  Event   271116 non-null  string[pyarrow]
 14  Medal   39783 non-null   string[pyarrow]
dtypes: double[pyarrow](1), int64[pyarrow](4), string[pyarrow](10)
memory usage: 46.4 MB


In [14]:
# note how much faster the masking compound expression becomes

#for comparison this was the old timing with numpy backend
# 32.4 ms ± 65.9 µs 
# 12.3 ms ± 41.3 µs

%timeit os[(os["Sex"] == "F") & (os["Height"] > 180) & (os["NOC"] == "SWE")]
%timeit os.query("Sex == 'F' & Height > 180 & NOC == 'SWE'") 


6.91 ms ± 163 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
16.6 ms ± 310 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
df1 = df1.convert_dtypes(dtype_backend="pyarrow")
df2 = df2.convert_dtypes(dtype_backend="pyarrow")
df3 = df3.convert_dtypes(dtype_backend="pyarrow")
df4 = df4.convert_dtypes(dtype_backend="pyarrow")

In [ ]:
# now we test our df additions and evals

# timing using numpy backend
# 1.74 s ± 47.8 ms 
# 1.79 s ± 148 ms 

# 1.09 s ± 227 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
%timeit sum_plain = df1+df2+df3+df4

# the eval ran for too long time using pyarrow so I stopped it
%timeit sum_eval = pd.eval("df1 + df2 + df3 + df4")


<div style="max-width:66ch;">

## Summary

In this lecture we've started working with more performant pandas methods such as eval and query. Then we went on to change the backend to the more language agnostic pyarrow for storing arrays instead of numpy which requires some computational overhead when changing from different languages. This improved the performance significantly when using pandas normal functions, expressions and operators for doing computations. 

Note however that you shouldn't optimize the code just for the optimization sake, unless there is a need for it. Rather choose more readable alternative than to optimize a few milliseconds unless the application requires it. 

</div>

<div style="background-color: #FFF; color: #212121; border-radius: 1px; width:22ch; box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px; display: flex; justify-content: center; align-items: center;">
<div style="padding: 1.5em 0; width: 70%;">
    <h2 style="font-size: 1.2rem;">Kokchun Giang</h2>
    <a href="https://www.linkedin.com/in/kokchungiang/" target="_blank" style="display: flex; align-items: center; gap: .4em; color:#0A66C2;">
        <img src="https://content.linkedin.com/content/dam/me/business/en-us/amp/brand-site/v2/bg/LI-Bug.svg.original.svg" width="20"> 
        LinkedIn profile
    </a>
    <a href="https://github.com/kokchun/Portfolio-Kokchun-Giang" target="_blank" style="display: flex; align-items: center; gap: .4em; margin: 1em 0; color:#0A66C2;">
        <img src="https://github.githubassets.com/images/modules/logos_page/GitHub-Mark.png" width="20"> 
        Github portfolio
    </a>
    <span>AIgineer AB</span>
<div>
</div>
